In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import holoviews as hv
from holoviews import opts
import hvplot.xarray

from pathlib import Path
from tqdm.notebook import tqdm

import networkx as nx
import geonetworkx as gnx

import warnings

from reservoirnetwork.network import ReservoirNetwork

warnings.filterwarnings('ignore')
hv.extension('bokeh')

We want to simulate a two reservoir system

# Two reservoir problem - checking water mass balance

The code below
- creates synthetic data - three inflow pulses of 1000 units to the upstream dam.
- simulates the outflow from the upstream dam assuming a linear reservoir.
- uses the outflow from upstream reservoir as inflow to the downstream reservoir.
- saves the data as "simulated", to be used in the next section to then run the river regulation model.

In [2]:
# define network
G = nx.DiGraph()
G.add_nodes_from([
    (1, {"name": "A"}),
    (2, {"name": "B"}),
])
G.add_edge(1, 2, travel_time=2)

start_time = pd.to_datetime('2000-01-01')
time_steps = 1e3

natural_runoff_data = xr.DataArray(
    data=np.array([[100., 0.] for _ in np.arange(time_steps)]),
    dims=['time', 'node'],
    coords={'node': [1, 2], 'time': pd.date_range(start_time, periods=time_steps, freq='1D')}
)
natural_runoff_data.loc[dict(time=slice('2000-05-01', '2000-05-30'), node=1)] = 1e3
natural_runoff_data.loc[dict(time=slice('2000-10-01', '2000-10-30'), node=1)] = 1e3
natural_runoff_data.loc[dict(time=slice('2000-03-01', '2000-03-30'), node=1)] = 1e3

forcings = xr.Dataset(
    data_vars={
        'natural_runoff': natural_runoff_data,
    }
)

data = xr.Dataset(
    coords={
        'node': [1, 2],
        'time': pd.date_range(start_time, periods=1, freq='1D')
    }
)

reservoir_network = ReservoirNetwork(G, start_time)

for timestep in tqdm(np.arange(time_steps)):
    reservoir_network.update(forcings, 1, algorithm='hydraulic', reservoir_algorithm='linear_reservoir')

reservoir_network.data

  0%|          | 0/1000 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:                     (node: 2, time: 1000)
Coordinates:
  * node                        (node) int64 1 2
  * time                        (time) datetime64[ns] 2000-01-01 ... 2002-09-26
Data variables:
    inflow                      (node, time) float64 100.0 100.0 ... 100.3 100.3
    outflow                     (node, time) float64 100.0 100.0 ... 102.2 102.2
    regulated_runoff            (node, time) float64 0.0 0.0 0.0 ... 100.3 100.3
    natural_runoff              (time, node) float64 100.0 0.0 ... 100.0 0.0
    theoretical_natural_runoff  (node, time) float64 100.0 100.0 ... 100.0 100.0
    storage                     (node, time) float64 1e+04 1e+04 ... 1.022e+04
    storage_change              (node, time) float64 0.0 0.0 ... -1.929 -1.913
    regulation                  (node, time) float64 0.0 0.0 ... -0.2931 -0.2902

In [3]:
ds = reservoir_network.data

ds.hvplot(
    x='time',
    y='theoretical_natural_runoff',
    by='node'
)

:NdOverlay   [node]
   :Curve   [time]   (theoretical_natural_runoff)

In [4]:
# save results
savefp = "../../data-two-reservoirs/simulated/pulse_linear_reservoir.nc"
reservoir_network.data.to_netcdf(savefp)

In [5]:
results_hvds = hv.Dataset(reservoir_network.data)
inflow_curve = results_hvds.to(hv.Curve, 'time', 'inflow', 'node').layout('node').cols(1).opts(title='inflow')
outflow_curve = results_hvds.to(hv.Curve, 'time', 'outflow', 'node').layout('node').cols(1).opts(title='outflow')
storage_curve = results_hvds.to(hv.Curve, 'time', 'storage', 'node').layout('node').cols(1).opts(title='storage')

(inflow_curve + outflow_curve + storage_curve).opts(opts.Curve(axiswise=False, tools=['hover']))

:Layout
   .NdLayout.I   :NdLayout   [node]
      :Curve   [time]   (inflow)
   .NdLayout.II  :NdLayout   [node]
      :Curve   [time]   (outflow)
   .NdLayout.III :NdLayout   [node]
      :Curve   [time]   (storage)

## we will now run the `wb` algorithm

The code below
- reads in the synthetic data created in the last section
- runs the river regulation model using the synthetic data to infer the inflow
- compares the inferred and synthetic data, plots, checks for water balance

In [22]:
## RUN MODEL
start_time = pd.to_datetime('2000-01-01')
time_steps = 1e3

# define network
G = nx.DiGraph()
G.add_nodes_from([
    (1, {"name": "A"}),
    (2, {"name": "B"}),
])
G.add_edge(1, 2, travel_time=2)

sim_data = xr.open_dataset('../../data-two-reservoirs/simulated/pulse_linear_reservoir.nc')
forcings = xr.Dataset(
    data_vars={
        'storage_change': sim_data['storage_change'],
        'theoretical_natural_runoff': sim_data['theoretical_natural_runoff']
    }
)

reservoir_network = ReservoirNetwork(G, start_time)

for timestep in tqdm(np.arange(time_steps)):
    reservoir_network.update(forcings, 1, algorithm='wb')

reservoir_network.data

  0%|          | 0/1000 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:                     (node: 2, time: 1000)
Coordinates:
  * node                        (node) int64 1 2
  * time                        (time) datetime64[ns] 2000-01-01 ... 2002-09-26
Data variables:
    inflow                      (node, time) float64 100.0 100.0 ... 100.3 100.3
    outflow                     (node, time) float64 100.0 100.0 ... 102.2 102.2
    regulated_runoff            (node, time) float64 0.0 0.0 0.0 ... 100.3 100.3
    natural_runoff              (node, time) float64 100.0 100.0 ... 0.0 0.0
    theoretical_natural_runoff  (node, time) float64 ...
    storage                     (node, time) float64 nan nan nan ... nan nan nan
    storage_change              (node, time) float64 ...
    regulation                  (node, time) float64 0.0 0.0 ... -0.2931 -0.2902

In [94]:
ds = reservoir_network.data
ds

<xarray.Dataset>
Dimensions:                     (node: 2, time: 1000)
Coordinates:
  * node                        (node) int64 1 2
  * time                        (time) datetime64[ns] 2000-01-01 ... 2002-09-26
Data variables:
    inflow                      (node, time) float64 100.0 100.0 ... 100.3 100.3
    outflow                     (node, time) float64 100.0 100.0 ... 102.2 102.2
    regulated_runoff            (node, time) float64 0.0 0.0 0.0 ... 100.3 100.3
    natural_runoff              (node, time) float64 100.0 100.0 ... 0.0 0.0
    theoretical_natural_runoff  (node, time) float64 ...
    storage                     (node, time) float64 nan nan nan ... nan nan nan
    storage_change              (node, time) float64 ...
    regulation                  (node, time) float64 0.0 0.0 ... -0.2931 -0.2902

In [95]:
# PLOT
upstream_downstream = {
    1: 'Upstream',
    2: 'Downstream'
}

simulated_hvds = hv.Dataset(ds)

inflow_plot = simulated_hvds.to(hv.Curve, 'time', 'inflow', 'node').layout('node').cols(1).opts(opts.Curve(xaxis='bare'))

inflow_plot

:NdLayout   [node]
   :Curve   [time]   (inflow)

In [96]:
outflow_plot = simulated_hvds.to(hv.Curve, 'time', 'outflow', 'node').layout('node').cols(1).opts(title='outflow')

outflow_plot

:NdLayout   [node]
   :Curve   [time]   (outflow)

Save the plots individually and organize them in figma

In [64]:
node1_ds = hv.Dataset(ds.sel(node=1))

inflow_curve_1 = node1_ds.to(hv.Curve, 'time', 'inflow', 'node', label='inflow').opts(title='Inflow to Node 1', xaxis='bare', ylabel='Inflow')
inflow_curve_1

:Curve   [time]   (inflow)

In [76]:
outflow_curve_1 = node1_ds.to(hv.Curve, 'time', 'outflow', 'node', label='Outflow').opts(title='Outflow from Node 1', xaxis='bare', ylabel='Outflow', ylim=(0, 1000))
outflow_curve_1

:Curve   [time]   (outflow)

In [77]:
node2_ds = hv.Dataset(ds.sel(node=2))

inflow_curve_2 = node2_ds.to(hv.Curve, 'time', 'inflow', 'node', label='Outflow').opts(title='Inflow to Node 2', xaxis='bare', ylabel='Inflow', ylim=(0, 1000))
inflow_curve_2

:Curve   [time]   (inflow)

In [78]:
outflow_curve_2 = node2_ds.to(hv.Curve, 'time', 'outflow', 'node', label='Outflow').opts(title='Outflow from Node 2', xaxis='bare', ylabel='Outflow', ylim=(0, 1000))
outflow_curve_2

:Curve   [time]   (outflow)

In [97]:
from bokeh.io import export_svgs

# https://stackoverflow.com/a/65267737/4091712
def export_svg(obj, filename):
    plot_state = hv.renderer('bokeh').get_plot(obj).state
    plot_state.output_backend = 'svg'
    export_svgs(plot_state, filename=filename)

In [80]:
export_svg(inflow_curve_1, '../../results/two_reservoirs/node1_inflow.svg')
export_svg(inflow_curve_2, '../../results/two_reservoirs/node2_inflow.svg')
export_svg(outflow_curve_1, '../../results/two_reservoirs/node1_outflow.svg')
export_svg(outflow_curve_2, '../../results/two_reservoirs/node2_outflow.svg')

## Does the water balance close?

In [81]:
simulated_ds = xr.open_dataset('/water2/pdas47/2023_01_24-river-regulation/data-two-reservoirs/simulated/pulse_outlet.nc')
inferred_ds = xr.open_dataset('/water2/pdas47/2023_01_24-river-regulation/data-two-reservoirs/inferred/pulse_outlet.nc')

np.allclose(simulated_ds['inflow'], inferred_ds['inflow'])

True

In [93]:
# check if the volume of inflow is the same
np.isclose(simulated_ds['inflow'].sum(), inferred_ds['inflow'].sum(), rtol=1e-5)

True